## Imports

In [ ]:
import os
import math

import numpy as np
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns
sns.set()

In [ ]:
import mixtureofconcave as subm
import determinantal as logsubm
import plottingtools

## Plottingtools

In [ ]:
def plot_objective_values(eval_objectives, eval_ground, setcolor, groundcolor=None):
    """
    As we greedily build out the set,
    we plot the value of the objective f(S_t)
    at every stage t
    :param eval_objectives: f(S_t) for 0 \leq t < k
    :param eval_ground: f(V)
    :param setcolor: color to plot f(S_t) with
    :param groundcolor: color to plot f(V) with
    """
    k = len(eval_objectives)
    plt.plot(np.arange(k), eval_objectives, "o", c=setcolor, label="$f(S_t)$")
    if groundcolor is not None:
        plt.plot(np.arange(k), [eval_ground,]*k, "--", c=groundcolor, label="$f(V)$")
    plt.xlabel("$|S_t|$")
    plt.legend(loc=2)

In [ ]:
def plot_membership_histogram(
    memberships, budgets,
    S,
    setcolor, groundcolor,
    budgetlabel="budget", setlabel="selection", value=-100
):
    """ Given a membership assignment matrix (n x p) and optionally group budgets (min or max),
        A selection of indices S of size k < n
        Plot the selection's group-wise distribution
        :param memberships: n rows (members), p columns (0 or 1 membership bools)
        :param budgets: indices in [n], length k
        :param setcolor: color to plot f(S_t) with
        :param groundcolor: color to plot f(V) with
    """
    
    [n, p] = memberships.shape
    k = len(S)
    
    Vcounts = np.sum(memberships, axis=0)
    Scounts = np.sum(memberships[S], axis=0)
    if budgetlabel == "quotas":
        assert budgets is not None
        unmet = np.where(Scounts < budgets)[0]
    elif budgetlabel == "capacities":
        assert budgets is not None
        unmet = np.where(Scounts > budgets)[0]
    else:
        unmet = None
    
    plt.bar(np.arange(p), Vcounts, color=groundcolor, label="all")  # all members
    plt.bar(np.arange(p), Scounts, color=setcolor, label=setlabel)  # selected members
    if budgets is not None:
        plt.scatter(np.arange(p), budgets, c="white", s=16, zorder=8, label=budgetlabel)  # all budgets
    if unmet is not None:
        plt.scatter(unmet, budgets[unmet], c="red", s=2, zorder=10, label="unmet "+budgetlabel)  # unmet budgets
    plt.xlabel("groups")
    plt.ylabel("membership counts")
    if value >= 0:
        plt.title(f"objective value = {value}")
    plt.legend()


---

## Submodular Objective

In [ ]:
def concave_function(x):
    """
    \phi(x)
    """
    # options: modA**0.5, np.log(1+modA), (1-np.exp(-modA)), modA/(1+modA)
    return x**0.5


In [ ]:
class submodular_oracle:
    
    def __init__(self, concave_function, weights, X):
        """
        :param concave_function: function \phi
        :param weights: for linear combination of \phi(\sum_{i \in A} X_{ij})) (length m)
        :param X: feature matrix (n x m)
        """
        self.concave_function = concave_function
        self.weights = weights
        self.X = X
        self.n = X.shape[0]
        self.m = X.shape[1]
        self.set = []
        self.modular_values = np.zeros(m)
        self.current_objective = 0
    
    def add_element(self, idx):
        self.modular_values += X[idx,:]
        self.current_objective = np.dot(
            self.weights, self.concave_function(self.modular_values)
        )
        self.set.append(idx)
    
    def compute_gain(self, idx):
        new_modular_values = self.modular_values + X[idx,:]
        return np.dot(
            self.weights, self.concave_function(new_modular_values)
        ) - self.current_objective

    def compute_set_value(self, indices):
        modular_values = np.sum(X[indices, :], axis=0)
        return np.dot(
            self.weights, self.concave_function(modular_values)
        )

In [ ]:
def random_vanilla(k):
    """ 
    For a given ground set, a feature matrix and mixture weights which define the submodular objective,
    Returns the greedy selection and step-wise objective values
    Over the addition of k items
    :param k: total capacity
    """
    
    oracle = submodular_oracle(concave_function, weights, X)
    
    V = np.arange(n).tolist()
    objs = []
    objs.append(0)
    
    for kk in range(k):
        
        greedy_winner = np.random.choice(V)
        
        # move winner from V to A
        V.remove(greedy_winner)
        oracle.add_element(greedy_winner)
        objs.append(oracle.current_objective)
    
    return oracle.set, objs


In [ ]:
def greedy_vanilla(k):
    """ 
    For a given ground set, a feature matrix and mixture weights which define the submodular objective,
    Returns the greedy selection and step-wise objective values
    Over the addition of k items
    :param k: total capacity
    """
    
    oracle = submodular_oracle(concave_function, weights, X)
    
    V = np.arange(n).tolist()
    objs = []
    objs.append(0)
    
    for kk in range(k):
        
        gains = [oracle.compute_gain(ii) for ii in V]
        greedy_winner = V[np.argmax(gains)]
        
        # move winner from V to A
        V.remove(greedy_winner)
        oracle.add_element(greedy_winner)
        objs.append(oracle.current_objective)
    
    return oracle.set, objs


In [ ]:
def greedy_quota(memberships, unfilled_quotas, k, verbose=False):
    """
    :param memberships: n rows (members), p columns (0 or 1 membership bools)
    :param unfilled_quotas: indices in [n], numpy array of length p
    :param k: total capacity
    """
    oracle = submodular_oracle(concave_function, weights, X)

    assert memberships.shape[0] == X.shape[0]
    assert memberships.shape[1] == len(unfilled_quotas)
    [n, p] = memberships.shape
    
    V = np.arange(n).tolist()
    objs = []
    objs.append(0)
    
    for jj in range(p):
        if np.sum(memberships[:,jj]) < unfilled_quotas[jj]:
            print("Not enough members in group {}, infeasible problem.".format(jj))
            return None, None
    
    """ Quota-filling stage """
    
    unfilled_groups = np.where(np.array(unfilled_quotas)>0)[0]
    V_quota_candidates = [ii for ii in V if np.sum(memberships[ii][unfilled_groups]) > 0]
    
    while np.sum(unfilled_quotas) > 0:
        
        unfilled_groups = np.where(unfilled_quotas>0)[0]
        V_quota_candidates = [ii for ii in V if np.sum(memberships[ii][unfilled_groups]) > 0]
        if len(V_quota_candidates) <= 0:
            # we ran out of quota candidates
            break
        
        gains = [oracle.compute_gain(ii) for ii in V_quota_candidates]
        greedy_winner = V_quota_candidates[np.argmax(gains)]
        
        # move winner from V to A
        V.remove(greedy_winner)
        oracle.add_element(greedy_winner)
        objs.append(oracle.current_objective)
        
        # decrement unfilled quotas
        unfilled_quotas = unfilled_quotas - memberships[greedy_winner]
        unfilled_quotas.clip(0, k)
        
        if verbose:
            print("unfilled_quotas = ", unfilled_quotas)
    
    """ Regular greedy stage """
    
    for kk in range(k - len(oracle.set)):
        
        gains = [oracle.compute_gain(ii) for ii in V]
        greedy_winner = V[np.argmax(gains)]
        
        # move winner from V to A
        V.remove(greedy_winner)
        oracle.add_element(greedy_winner)
        objs.append(oracle.current_objective)
    
    return oracle.set, objs


In [ ]:
def greedy_capacity(memberships, unfilled_capacities, k, verbose=False):
    """
    :param memberships: n rows (members), p columns (0 or 1 membership bools)
    :param unfilled_capacities: indices in [n], numpy array of length p
    :param k: total capacity
    """
    oracle = submodular_oracle(concave_function, weights, X)

    assert memberships.shape[0] == X.shape[0]
    assert memberships.shape[1] == len(unfilled_capacities)
    [n, p] = memberships.shape
    
    V = np.arange(n).tolist()
    objs = []
    objs.append(0)
    
    unfilled_groups = np.where(np.array(unfilled_capacities)>0)[0]
    V_capacity_candidates = [ii for ii in V if np.sum(memberships[ii][unfilled_groups]) > 0]
    
    for kk in range(k):
        
        unfilled_groups = np.where(unfilled_capacities>0)[0]
        V_capacity_candidates = [ii for ii in V if np.sum(memberships[ii][unfilled_groups]) > 0]
        if len(V_capacity_candidates) <= 0:
            # we ran out of candidates
            return oracle.set, objs
        
        gains = [oracle.compute_gain(ii) for ii in V_capacity_candidates]
        greedy_winner = V_capacity_candidates[np.argmax(gains)]
        
        # move winner from V to A
        V.remove(greedy_winner)
        oracle.add_element(greedy_winner)
        objs.append(oracle.current_objective)
        
        # decrement unfilled quotas
        unfilled_capacities = unfilled_capacities - memberships[greedy_winner]
        unfilled_capacities.clip(0, k)
        
        if verbose:
            print("unfilled_capacities = ", unfilled_capacities)
    
    return oracle.set, objs


## आधी क्वोटोबा मग विठोबा

### Disjoint Memberships

In [ ]:
# Setup
n = 100
m = 80
k = 20
p = 3
p_dist = [0.4, 0.3, 0.3]  # across groups
disadvantage_factor = 0.7  # weights shrunk by
disadvantage_features = [0, 16, 24]  # number of features to shrink weights on

np.random.seed(0)
X = np.random.random((n, m))
weights = np.random.random(m); weights = weights/np.max(weights)

In [ ]:
# Disjoint Memberships
np.random.seed(1)
list_memberships = np.random.choice(np.arange(p), size=(n), p=p_dist)  # length n
disjoint_memberships = np.eye(p)[list_memberships].astype(int)
disjoint_quotas = np.array([4, 4, 4])
disjoint_capacities = np.array([12, 12, 12])

# Inversely correlate with utility
for pp in range(p):
    df = np.random.choice(np.arange(m), size=disadvantage_features[pp], replace=False)
    dm = np.where(list_memberships == pp)[0]
    for member in dm:
        for feature in df:
            X[member, feature] *= disadvantage_factor

In [ ]:
# # Intersecting Memberships
# np.random.seed(1)
# intersecting_memberships = np.random.choice([0, 1], size=(n, p))
# intersecting_quotas = np.array([2, 2, 2])

In [ ]:
# Color Scheme
# 'dimgrey' '#696969'
# 'darkkhaki' '#BDB76B'  # ground set
# 'indianred' '#CD5C5C'  # random
# 'darkcyan' '#008B8B'  # unconstrained
# 'gold' '#FFD700'  # disjoint membership capacity
# 'skyblue' '#87CEEB'  # intersecting membership capacity
# 'orange' '#FFA500'  # disjoint membership quota
# 'lightpink' '#FFB6C1'  # intersecting membership quota

<p style="background-color:#008B8B">
<span style="color:white">Unconstrained Submodular Max</span>.
</p>

In [ ]:
## Ground set eval
oracle = submodular_oracle(concave_function, weights, X)
ground = oracle.compute_set_value(np.arange(n))

In [ ]:
## Random choice
S_random, objectives_random = random_vanilla(k)

# [print(oo) for oo in objectives_random]
# print("group memberships = ", np.sum(disjoint_memberships[S_random], axis=0))
# plt.figure(figsize=(10,5))
# plt.subplot(1,2,1)
# plot_objective_values(objectives_random, ground, "indianred", "dimgrey")
# plt.subplot(1,2,2)
# plot_membership_histogram(
#     disjoint_memberships, None, S_random,
#     "indianred", "darkkhaki", setlabel="random",
#     value=objectives_random[-1]
# )

In [ ]:
## Submod greedy
S_unc, objectives_unc = greedy_vanilla(k)

# [print(oo) for oo in objectives_unc]
# print("group memberships = ", np.sum(disjoint_memberships[S_unc], axis=0))
# plt.figure(figsize=(10,5))
# plt.subplot(1,2,1)
# plot_objective_values(objectives_unc, ground, "darkcyan", "dimgrey")
# plt.subplot(1,2,2)
# plot_membership_histogram(
#     disjoint_memberships, None, S_unc,
#     "darkcyan", "darkkhaki", setlabel="unconstrained",
#     value=objectives_unc[-1]
# )

<p style="background-color:#lightpink">
<span style="color:white">Constrained Submodular Max with Intersecting Membership Quota</span>.
</p>

In [ ]:
## Quota greedy (Disjoint)
S_quota, objectives_quota = greedy_quota(disjoint_memberships, disjoint_quotas, k)

# [print(oo) for oo in objectives]
# print("group memberships = ", np.sum(disjoint_memberships[S_quota], axis=0))
# plt.figure(figsize=(10,5))
# plt.subplot(1,2,1)
# plot_objective_values(objectives_quota, ground, "darkorange", "dimgrey")
# plt.subplot(1,2,2)
# plot_membership_histogram(
#     disjoint_memberships, disjoint_quotas, S_quota,
#     "darkorange", "darkkhaki", setlabel="quota greedy",
#     budgetlabel="quotas", value=objectives_quota[-1]
# )

In [ ]:
## Capacity greedy (Disjoint)
S_capacity, objectives_capacity = greedy_capacity(disjoint_memberships, disjoint_capacities, k)

# [print(oo) for oo in objectives]
# print("group memberships = ", np.sum(disjoint_memberships[S_capacity], axis=0))
# plt.figure(figsize=(10,5))
# plt.subplot(1,2,1)
# plot_objective_values(objectives_capacity, ground, "darkorange", "dimgrey")
# plt.subplot(1,2,2)
# plot_membership_histogram(
#     disjoint_memberships, disjoint_capacities, S_capacity,
#     "gold", "darkkhaki", setlabel="quota greedy",
#     budgetlabel="capacities", value=objectives_capacity[-1]
# )

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(2,2,1)
plot_membership_histogram(
    disjoint_memberships, None, S_random,
    "indianred", "darkkhaki", setlabel="random",
    value=objectives_random[-1]
)
plt.subplot(2,2,2)
plot_membership_histogram(
    disjoint_memberships, None, S_unc,
    "darkcyan", "darkkhaki", setlabel="unconstrained",
    value=objectives_unc[-1]
)
plt.subplot(2,2,3)
plot_membership_histogram(
    disjoint_memberships, disjoint_quotas, S_quota,
    "darkorange", "darkkhaki", setlabel="quota greedy",
    budgetlabel="quotas", value=objectives_quota[-1]
)
plt.subplot(2,2,4)
plot_membership_histogram(
    disjoint_memberships, disjoint_capacities, S_capacity,
    "gold", "darkkhaki", setlabel="quota greedy",
    budgetlabel="capacities", value=objectives_capacity[-1]
)

---